In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from typing import Tuple
import joblib
from typing import List

In [2]:
def calc_rul(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the remaining useful life (RUL) for each engine 
    """
    df['RUL'] = (df.sort_values(['time'], ascending=False)
                    .groupby(['unit_number']).cumcount())
    return df

def gen_col_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generates the columns for training data set
    """
    cols = (['unit_number', 'time'] 
            + [f'op_setting_{i+1}' for i in range(3)]
            + [f'sensor_{i+1}' for i in range(df.shape[1] - 5)])
    df.columns = cols
    return df

def drop_empty_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops empty columns from dataframe
    """
    na_by_col = df.isna().sum()
    row_cnt = df.shape[0]
    cols_to_keep = [True if na_cnt != row_cnt else False for na_cnt in na_by_col]
    return df.iloc[:, cols_to_keep]


def load_ts_data(file_path: str) -> pd.DataFrame:
    """
    Loads time series data
    Args:
        - drive_path (str): path to the file
    Returns:
        - pandas dataframe containing data with expected column names
    """
    df = pd.read_csv(file_path, sep=' ', header=None)
    df = (df.pipe(drop_empty_cols)
            .pipe(gen_col_names))
    return df


def drop_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops columns that will not be utilized in the final model
    """
    SENSOR_COLS_TO_DROP = [f'sensor_{i}' for i in (1, 5, 6, 10, 16, 18, 19)]
    SETTING_COLS_TO_DROP = [f'op_setting_{i}' for i in range(1, 3+1)]
    COLS_TO_DROP = SENSOR_COLS_TO_DROP + SETTING_COLS_TO_DROP + ['unit_number', 'time']
    return df.drop(columns=COLS_TO_DROP)


def drop_columns_(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops columns that will not be utilized in the final model
    """
    SENSOR_COLS_TO_DROP = [f'sensor_{i}' for i in (1, 5, 6, 10, 16, 18, 19)]
    SETTING_COLS_TO_DROP = [f'op_setting_{i}' for i in range(1, 3+1)]
    COLS_TO_DROP = SENSOR_COLS_TO_DROP + SETTING_COLS_TO_DROP + ['time']
    return df.drop(columns=COLS_TO_DROP)


def add_lag_feature(df: pd.DataFrame, **kwargs):
    """
    Create a lagged version of features (*args) in the df
    """
    required_params = ['lag_level', 'features']
    for param in required_params:
        if param not in kwargs.keys():
            raise Exception(f'Required param "{param}" does not exist in function call!'
                            f'{required_params}')
    # gather required params
    lag_level = kwargs['lag_level']
    features = kwargs['features']
    
    for feature in features:
        new_lag_feature: str = f'{feature}_lag_{lag_level}'
        df[new_lag_feature] = df[feature].shift(lag_level)
    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleaning input data before training or inference;
    dropping columns that do not have much predictive power; 
    see analysis described below:
    https://towardsdatascience.com/predictive-maintenance-of-turbofan-engines-ec54a083127
    """
    df = df.pipe(drop_columns)
    lag_features = [col for col in df.columns.tolist() if col != 'RUL']
    df = (df.pipe(add_lag_feature, **{'lag_level': 1, 'features': lag_features})
            .dropna())
    return df

In [3]:
# load first training file
DATA_DIR = '../data/raw/CMAPSSData_small'
TRAIN_TS_FILE = 'train_FD001.txt'
TEST_TS_FILE = 'test_FD001.txt'
TEST_RUL_FILE = 'RUL_FD001.txt'
train_df = load_ts_data(f'{DATA_DIR}/{TRAIN_TS_FILE}')
test_df = load_ts_data(f'{DATA_DIR}/{TEST_TS_FILE}')
y_test = pd.read_csv(f'{DATA_DIR}/{TEST_RUL_FILE}', header=None)
y_test.columns = ['rul']
train_df = (train_df
                .pipe(calc_rul)
                .pipe(preprocess))
# clip RUL
train_df['RUL'] = train_df['RUL'].clip(upper=125)
train_df.head()

,sensor_2,sensor_3,sensor_4,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_14,...,sensor_8_lag_1,sensor_9_lag_1,sensor_11_lag_1,sensor_12_lag_1,sensor_13_lag_1,sensor_14_lag_1,sensor_15_lag_1,sensor_17_lag_1,sensor_20_lag_1,sensor_21_lag_1
1,642.15,1591.82,1403.14,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,...,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392.0,39.06,23.4190
2,642.35,1587.99,1404.20,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,...,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392.0,39.00,23.4236
3,642.35,1582.79,1401.87,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,...,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390.0,38.95,23.3442
4,642.37,1582.85,1406.22,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,...,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392.0,38.88,23.3739
5,642.10,1584.47,1398.37,554.67,2388.02,9049.68,47.16,521.68,2388.03,8132.85,...,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393.0,38.90,23.4044


In [4]:
test_df.head()

,unit_number,time,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [5]:
test_df.columns.tolist()

['unit_number',
 'time',
 'op_setting_1',
 'op_setting_2',
 'op_setting_3',
 'sensor_1',
 'sensor_2',
 'sensor_3',
 'sensor_4',
 'sensor_5',
 'sensor_6',
 'sensor_7',
 'sensor_8',
 'sensor_9',
 'sensor_10',
 'sensor_11',
 'sensor_12',
 'sensor_13',
 'sensor_14',
 'sensor_15',
 'sensor_16',
 'sensor_17',
 'sensor_18',
 'sensor_19',
 'sensor_20',
 'sensor_21']

In [6]:
train_df.describe()

,sensor_2,sensor_3,sensor_4,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_14,...,sensor_8_lag_1,sensor_9_lag_1,sensor_11_lag_1,sensor_12_lag_1,sensor_13_lag_1,sensor_14_lag_1,sensor_15_lag_1,sensor_17_lag_1,sensor_20_lag_1,sensor_21_lag_1
count,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,...,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000,20630.000000
mean,642.680975,1590.523159,1408.934186,553.367663,2388.096653,9065.243864,47.541172,521.413458,2388.096156,8143.752971,...,2388.096644,9065.243123,47.541136,521.413572,2388.096144,8143.753033,8.442143,393.210519,38.816292,23.289717
std,0.500029,6.131295,9.000636,0.885087,0.070987,22.083016,0.267093,0.737569,0.071919,19.076605,...,0.070978,22.083399,0.267054,0.737424,0.071912,19.076586,0.037504,1.548679,0.180724,0.108241
min,641.210000,1571.040000,1382.250000,549.850000,2387.900000,9021.730000,46.850000,518.690000,2387.880000,8099.940000,...,2387.900000,9021.730000,46.850000,518.690000,2387.880000,8099.940000,8.324900,388.000000,38.140000,22.894200
25%,642.330000,1586.260000,1402.360000,552.810000,2388.050000,9053.100000,47.350000,520.960000,2388.040000,8133.242500,...,2388.050000,9053.100000,47.350000,520.960000,2388.040000,8133.242500,8.414900,392.000000,38.700000,23.221800
50%,642.640000,1590.100000,1408.040000,553.440000,2388.090000,9060.660000,47.510000,521.480000,2388.090000,8140.540000,...,2388.090000,9060.660000,47.510000,521.480000,2388.090000,8140.540000,8.438900,393.000000,38.830000,23.297900
75%,643.000000,1594.380000,1414.557500,554.010000,2388.140000,9069.420000,47.700000,521.950000,2388.140000,8148.310000,...,2388.140000,9069.420000,47.700000,521.950000,2388.140000,8148.310000,8.465575,394.000000,38.950000,23.366800
max,644.530000,1616.910000,1441.490000,556.060000,2388.560000,9244.590000,48.530000,523.380000,2388.560000,8293.720000,...,2388.560000,9244.590000,48.530000,523.380000,2388.560000,8293.720000,8.584800,400.000000,39.430000,23.618400


In [7]:
# drop rows with NA values
train_df = train_df.dropna()
y_train = train_df['RUL'].values
X_train = train_df.drop(columns=['RUL'])

model = LinearRegression(normalize=True)

In [8]:
pipeline = make_pipeline(model)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('linearregression', LinearRegression(normalize=True))])

In [9]:
pipeline.named_steps

{'linearregression': LinearRegression(normalize=True)}

In [10]:
test_df.head()

,unit_number,time,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [11]:
# TODO: make sure dropping unit_number occurs after lagged features are added (drop other useless features first)
# but need to keep 'unit_number' for the partition to find latest time step record

# only want to make predictions based on the last timestamp (cycle) for each engine in test set
test = test_df.pipe(drop_columns_)
lag_features: List[str] = [col for col in test.columns.tolist() if col not in ('RUL', 'unit_number',)]
test = test.pipe(add_lag_feature, **{'lag_level': 1, 'features': lag_features})
test = (test.groupby('unit_number')
            .last()
            .reset_index()
            .drop(columns=['unit_number']))
predictions = np.floor(pipeline.predict(test))

In [12]:
test_mse = mean_squared_error(y_test, predictions, squared=False)
print(f'Test Set MSE: {test_mse}')

Test Set MSE: 21.641857591251267


In [13]:
y_test[:10]

,rul
0,112
1,98
2,69
3,82
4,91
5,93
6,91
7,95
8,111
9,96


In [14]:
predictions[:10]

array([122., 100.,  73.,  82.,  89.,  95., 101.,  79.,  94.,  94.])

In [15]:
y_test[-10:]

,rul
90,38
91,20
92,85
93,55
94,128
95,137
96,82
97,59
98,117
99,20


In [16]:
predictions[-10:]

array([ 48.,  38.,  52.,  62., 109., 122.,  88.,  78., 122.,  15.])

In [17]:
# store serialized pipeline
pipeline_name = 'time_series_regression_clipped_pipeline.gz'
file_path = f'../models/{pipeline_name}'
joblib.dump(pipeline, file_path)

['../models/time_series_regression_clipped_pipeline.gz']